In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, Embedding, Input, Reshape, Concatenate
from keras import metrics
from keras.optimizers import Adam, rmsprop
from keras.callbacks import TensorBoard, ReduceLROnPlateau
from time import time
from IPython.display import display

Using TensorFlow backend.


In [2]:
experiment_name='mlhw-mlp-with-preprocess'

## Load Dataset

In [11]:
data_val = pd.read_csv('dataset/train_values.csv')
data_labels = pd.read_csv('dataset/train_labels.csv')

# Merge values and label
data = data_val.merge(data_labels, on='patient_id')
data = data.drop(columns='patient_id')

# Convert string values to integers
data["thal"] = data["thal"].astype('category')
data["thal"] = data["thal"].cat.codes

# Split dataset
train = data
# train = data.sample(frac=0.8)
# validate = data.sample(frac=0.2)

xsize = train.shape[1] - 1
ysize = 1
label_column = 'heart_disease_present'
cols = list(train.columns)
cols.remove(label_column)



In [12]:
display(train.head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present
0,1,1,128,2,0,0,2,308,0.0,1,45,170,0,0
1,2,1,110,3,0,0,0,214,1.6,0,54,158,0,0
2,1,1,125,4,3,0,2,304,0.0,1,77,162,1,1
3,1,2,152,4,0,0,0,223,0.0,1,40,181,0,1
4,3,2,178,1,0,0,2,270,4.2,1,59,145,0,0


## Preprocessing

### Data Whitening
This will normalize our values so higher order values won't dominate other values

In [13]:
train_mean = train[cols].mean(axis=0)
train_std = train[cols].std(axis=0)
train[cols] = (train[cols] - train_mean) / train_std
validate[cols] = (validate[cols] - train_mean) / train_std

display(train[cols].head())

,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,-0.888762,-0.645259,-0.194652,-1.231340,-0.716404,-0.437019,0.951196,1.115158,-0.900694,0.670152,-1.051032,0.929891,-0.678852
1,0.727169,-0.645259,-1.252825,-0.165757,-0.716404,-0.437019,-1.051322,-0.667915,0.526148,-1.483908,-0.086892,0.386007,-0.678852
2,-0.888762,-0.645259,-0.371014,0.899825,2.378462,-0.437019,0.951196,1.039283,-0.900694,0.670152,2.377024,0.567302,1.464891
3,-0.888762,1.114538,1.216246,0.899825,-0.716404,-0.437019,-1.051322,-0.497195,-0.900694,0.670152,-1.586666,1.428452,-0.678852
4,2.343100,1.114538,2.744719,-2.296923,-0.716404,-0.437019,0.951196,0.394342,2.844768,0.670152,0.448742,-0.203201,-0.678852


### Re-arrange Data for Categorical and Continous Variables

In [14]:

cat_cols = [
    'thal',
    'chest_pain_type',
    'num_major_vessels',
    'fasting_blood_sugar_gt_120_mg_per_dl',
    'resting_ekg_results',
    'sex',
    'exercise_induced_angina'
]
cont_cols = [
    'slope_of_peak_exercise_st_segment',
    'resting_blood_pressure',
    'serum_cholesterol_mg_per_dl',
    'oldpeak_eq_st_depression',
    'age',
    'max_heart_rate_achieved',
]


def preproc_input(input_data):
    input_list = []
    for cat_col in cat_cols:
        input_list.append(input_data[cat_col].values)

    input_list.append(input_data[cont_cols].values)
    return input_list
    

## Define Model 

In [23]:
inputs = []
embeddings = []

# For Categorical Variables
for cat_col in cat_cols :
     
    no_of_unique_cat  = train[cat_col].nunique()
    embedding_size = min(np.ceil((no_of_unique_cat)/2), 50)
    embedding_size = int(embedding_size)
    vocab  = no_of_unique_cat + 1
    
    emb_input = Input(shape=(1,))
    emb = Embedding(vocab ,embedding_size, input_length = 1)(emb_input)
    emb = Reshape(target_shape=(embedding_size, ))(emb)
    inputs.append(emb_input)
    embeddings.append(emb)

# Continuous variables
input_cont = Input(shape=(len(cont_cols),))
embedding_cont = Dense(len(cont_cols))(input_cont) 
inputs.append(input_cont)
embeddings.append(embedding_cont)

x = Concatenate()(embeddings)
x = Dense(80)(x)
x = Activation('relu')(x)
x = Dropout(.35)(x)
x = BatchNormalization()(x)

x = Dense(20)(x)
x = Activation('relu')(x)
x = Dropout(.15)(x)
x = BatchNormalization()(x)

x = Dense(10)(x)
x = Activation('relu')(x)
x = Dropout(.15)(x)
x = BatchNormalization()(x)

x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(
    inputs=inputs, 
    outputs=[output]
)

initial_lr = 0.001

# opt_name = 'rmsprop'
# opt = rmsprop(lr=initial_lr, decay=1e-6)

opt_name='adam'
opt = Adam(lr=initial_lr)


model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=[metrics.binary_accuracy]
)

## Training

In [24]:
epochs = 500
batch_size = 32
exp_stamp = '{}-ilr={}-opt={}-{}'.format(experiment_name, initial_lr, opt_name, time())

tb_cb = TensorBoard(
    batch_size=batch_size,
    log_dir="logs/{}".format(exp_stamp),
    histogram_freq=0,
    write_graph=True,
    write_images=True,
    update_freq='batch'
)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [tb_cb, lr_reducer]


x_train = preproc_input(train)
y_train = train[label_column].values
x_val = preproc_input(validate)
y_val = validate[label_column].values

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    callbacks=callbacks
)

# Save model and weights
save_dir=os.path.join(os.getcwd(), 'saved_models')
model_name= '{}.h5'.format(exp_stamp)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Epoch 1/500
180/180 [==============================] - 1s 7ms/step - loss: 0.8240 - binary_accuracy: 0.5611
Epoch 2/500
180/180 [==============================] - 0s 402us/step - loss: 0.8578 - binary_accuracy: 0.5167
Epoch 3/500
180/180 [==============================] - 0s 423us/step - loss: 0.7990 - binary_accuracy: 0.5889
Epoch 4/500
180/180 [==============================] - 0s 528us/step - loss: 0.7046 - binary_accuracy: 0.6222
Epoch 5/500
180/180 [==============================] - 0s 469us/step - loss: 0.6288 - binary_accuracy: 0.6611
Epoch 6/500
180/180 [==============================] - 0s 564us/step - loss: 0.7085 - binary_accuracy: 0.6056
Epoch 7/500
180/180 [==============================] - 0s 409us/step - loss: 0.7291 - binary_accuracy: 0.5778
Epoch 8/500
180/180 [==============================] - 0s 465us/step - loss: 0.6649 - binary_accuracy: 0.6500
Epoch 9/500
180/180 [==============================] - 0s 535us/step - loss: 0.6382 - binary_accuracy: 0.6278
Epoch 10/500

180/180 [==============================] - 0s 445us/step - loss: 0.4087 - binary_accuracy: 0.8222
Epoch 75/500
180/180 [==============================] - 0s 449us/step - loss: 0.4203 - binary_accuracy: 0.8278
Epoch 76/500
180/180 [==============================] - 0s 423us/step - loss: 0.4326 - binary_accuracy: 0.8056
Epoch 77/500
180/180 [==============================] - 0s 389us/step - loss: 0.4019 - binary_accuracy: 0.8556
Epoch 78/500
180/180 [==============================] - 0s 356us/step - loss: 0.3848 - binary_accuracy: 0.8167
Epoch 79/500
180/180 [==============================] - 0s 360us/step - loss: 0.3816 - binary_accuracy: 0.8222
Epoch 80/500
180/180 [==============================] - 0s 384us/step - loss: 0.4012 - binary_accuracy: 0.8389
Epoch 81/500
180/180 [==============================] - 0s 377us/step - loss: 0.3979 - binary_accuracy: 0.8111
Epoch 82/500
180/180 [==============================] - 0s 389us/step - loss: 0.3952 - binary_accuracy: 0.8111
Epoch 83/500
1

180/180 [==============================] - 0s 373us/step - loss: 0.2350 - binary_accuracy: 0.9167
Epoch 148/500
180/180 [==============================] - 0s 361us/step - loss: 0.2884 - binary_accuracy: 0.8944
Epoch 149/500
180/180 [==============================] - 0s 345us/step - loss: 0.3506 - binary_accuracy: 0.8389
Epoch 150/500
180/180 [==============================] - 0s 349us/step - loss: 0.2948 - binary_accuracy: 0.8722
Epoch 151/500
180/180 [==============================] - 0s 375us/step - loss: 0.2845 - binary_accuracy: 0.8722
Epoch 152/500
180/180 [==============================] - 0s 384us/step - loss: 0.2882 - binary_accuracy: 0.8778
Epoch 153/500
180/180 [==============================] - 0s 393us/step - loss: 0.2722 - binary_accuracy: 0.8833
Epoch 154/500
180/180 [==============================] - 0s 407us/step - loss: 0.2148 - binary_accuracy: 0.9111
Epoch 155/500
180/180 [==============================] - 0s 394us/step - loss: 0.2929 - binary_accuracy: 0.8833
Epoch 

180/180 [==============================] - 0s 408us/step - loss: 0.2171 - binary_accuracy: 0.9167
Epoch 221/500
180/180 [==============================] - 0s 378us/step - loss: 0.1694 - binary_accuracy: 0.9389
Epoch 222/500
180/180 [==============================] - 0s 377us/step - loss: 0.2365 - binary_accuracy: 0.8944
Epoch 223/500
180/180 [==============================] - 0s 369us/step - loss: 0.1764 - binary_accuracy: 0.9389
Epoch 224/500
180/180 [==============================] - 0s 389us/step - loss: 0.1881 - binary_accuracy: 0.9500
Epoch 225/500
180/180 [==============================] - 0s 381us/step - loss: 0.1833 - binary_accuracy: 0.9444
Epoch 226/500
180/180 [==============================] - 0s 382us/step - loss: 0.2109 - binary_accuracy: 0.8889
Epoch 227/500
180/180 [==============================] - 0s 354us/step - loss: 0.2240 - binary_accuracy: 0.8944
Epoch 228/500
180/180 [==============================] - 0s 366us/step - loss: 0.1555 - binary_accuracy: 0.9389
Epoch 

180/180 [==============================] - 0s 360us/step - loss: 0.1529 - binary_accuracy: 0.9444
Epoch 294/500
180/180 [==============================] - 0s 414us/step - loss: 0.0874 - binary_accuracy: 0.9778
Epoch 295/500
180/180 [==============================] - 0s 359us/step - loss: 0.1132 - binary_accuracy: 0.9611
Epoch 296/500
180/180 [==============================] - 0s 356us/step - loss: 0.1642 - binary_accuracy: 0.9278
Epoch 297/500
180/180 [==============================] - 0s 381us/step - loss: 0.2005 - binary_accuracy: 0.9222
Epoch 298/500
180/180 [==============================] - 0s 349us/step - loss: 0.1104 - binary_accuracy: 0.9667
Epoch 299/500
180/180 [==============================] - 0s 356us/step - loss: 0.1374 - binary_accuracy: 0.9556
Epoch 300/500
180/180 [==============================] - 0s 358us/step - loss: 0.1143 - binary_accuracy: 0.9500
Epoch 301/500
180/180 [==============================] - 0s 389us/step - loss: 0.1457 - binary_accuracy: 0.9500
Epoch 

180/180 [==============================] - 0s 409us/step - loss: 0.1458 - binary_accuracy: 0.9222
Epoch 367/500
180/180 [==============================] - 0s 420us/step - loss: 0.0512 - binary_accuracy: 0.9889
Epoch 368/500
180/180 [==============================] - 0s 379us/step - loss: 0.0806 - binary_accuracy: 0.9611
Epoch 369/500
180/180 [==============================] - 0s 436us/step - loss: 0.0908 - binary_accuracy: 0.9778
Epoch 370/500
180/180 [==============================] - 0s 447us/step - loss: 0.1018 - binary_accuracy: 0.9500
Epoch 371/500
180/180 [==============================] - 0s 367us/step - loss: 0.1249 - binary_accuracy: 0.9556
Epoch 372/500
180/180 [==============================] - 0s 368us/step - loss: 0.1325 - binary_accuracy: 0.9444
Epoch 373/500
180/180 [==============================] - 0s 350us/step - loss: 0.1061 - binary_accuracy: 0.9667
Epoch 374/500
180/180 [==============================] - 0s 345us/step - loss: 0.1502 - binary_accuracy: 0.9444
Epoch 

180/180 [==============================] - 0s 366us/step - loss: 0.0722 - binary_accuracy: 0.9611
Epoch 440/500
180/180 [==============================] - 0s 379us/step - loss: 0.1019 - binary_accuracy: 0.9722
Epoch 441/500
180/180 [==============================] - 0s 363us/step - loss: 0.0840 - binary_accuracy: 0.9611
Epoch 442/500
180/180 [==============================] - 0s 402us/step - loss: 0.0984 - binary_accuracy: 0.9611
Epoch 443/500
180/180 [==============================] - 0s 386us/step - loss: 0.1305 - binary_accuracy: 0.9556
Epoch 444/500
180/180 [==============================] - 0s 387us/step - loss: 0.0692 - binary_accuracy: 0.9778
Epoch 445/500
180/180 [==============================] - 0s 353us/step - loss: 0.0773 - binary_accuracy: 0.9667
Epoch 446/500
180/180 [==============================] - 0s 375us/step - loss: 0.1122 - binary_accuracy: 0.9500
Epoch 447/500
180/180 [==============================] - 0s 365us/step - loss: 0.1048 - binary_accuracy: 0.9611
Epoch 

## Testing

### Preprocess Test Set

In [20]:
test = pd.read_csv('dataset/test_values.csv')

# Convert string values to integers
test["thal"] = test["thal"].astype('category')
test["thal"] = test["thal"].cat.codes

# Whiten test set
test[cols] = (test[cols] - train_mean) / train_std

display(test[cols].head())

x_test = preproc_input(test[cols])


,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,0.727169,1.114538,2.274420,-2.296923,-0.716404,-0.437019,0.951196,0.735781,-0.722339,0.670152,0.448742,0.431331,-0.678852
1,-0.888762,-0.645259,0.393222,0.899825,-0.716404,-0.437019,-1.051322,-1.255950,0.347793,-1.483908,-2.122300,1.473776,-0.678852
2,0.727169,1.114538,-0.664951,0.899825,-0.716404,-0.437019,0.951196,-1.369763,1.328747,0.670152,-1.265286,-1.336294,1.464891
3,-0.888762,-0.645259,-1.723124,-0.165757,0.315218,-0.437019,-1.051322,1.304847,-0.900694,-1.483908,0.555869,0.476654,-0.678852
4,0.727169,-0.645259,0.393222,0.899825,0.315218,-0.437019,0.951196,-1.578420,2.309702,0.670152,0.662996,-1.109675,1.464891


In [22]:
predictions = model.predict(x_test)

output = pd.DataFrame({
    'patient_id': test['patient_id'],
    'heart_disease_present': predictions[:,0]
})

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(output)

output_dir = 'output'
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)
output_file = output_dir + '/' + experiment_name + '-' + 'out.csv'
output.to_csv(output_file, index=False)

,patient_id,heart_disease_present
0,olalu7,0.756895
1,z9n6mx,0.000302
2,5k4413,0.999823
3,mrg7q5,0.004211
4,uki4do,0.998945
5,kev1sk,0.000805
6,9n6let,0.003094
7,jxmtyg,0.996848
8,51s2ff,0.328403
9,wi9mcs,0.102421
